In [1]:
import inro.modeller as m
import os
import glob
mm = m.Modeller()
eb = mm.emmebank
import inro.emme.matrix as _matrix
import pandas as pd
from copy import deepcopy

matrix_calculator = m.Modeller().tool("inro.emme.matrix_calculation.matrix_calculator")
create_matrix =  m.Modeller().tool("inro.emme.data.matrix.create_matrix")
change_matrix = m.Modeller().tool("inro.emme.data.matrix.change_matrix_properties") 
import_matrices = mm.tool("tmg.input_output.import_binary_matrix")  
matrix_transaction = mm.tool('inro.emme.data.matrix.matrix_transaction')
create_extra = m.Modeller().tool("inro.emme.data.extra_attribute.create_extra_attribute")
network_calc= m.Modeller().tool("inro.emme.network_calculation.network_calculator")

def clear_matrices(emmebank, exception_array = [""], type = "FULL"):
    for matrix in emmebank.matrices():
        if matrix.type == type:
            if matrix.id not in exception_array:
                emmebank.delete_matrix(matrix.id)

def import_matrices_from_directory(matrix_folder, matrix_list, extension, scenario):
    #imports all matrices in directory and subfolders of the specified extension type
    #matrix_list should be in the format:
    #   {file_name1: matrix_num1, file_name2: matrix_num2, ...}
    import os

    for root, dirs, files in os.walk(matrix_folder):
        for matrix_file in files:
            if matrix_file.endswith(extension):
                name = os.path.splitext(matrix_file)[0]
                if name in matrix_list:
                    matrix_number = int(matrix_list[name][2:])
                    import_matrices(4, matrix_number ,os.path.join(root, matrix_file),scenario, name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_") )
                    print "Imported matrix " + matrix_list[name]
                    change_matrix(matrix = matrix_list[name],
                        matrix_name = name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_")[:40],
                        matrix_description = name)
    
    #check
    for name in matrix_list:
        if eb.matrix(matrix_list[name]): 
            n_matrix = eb.matrix(matrix_list[name]).get_numpy_data(scenario)
            if n_matrix.sum() == 0 :
                print "Matrix %s was not imported, or contains no values!" % name
        else:
            print "Matrix %s was not imported!" % name
    print "Finished Importing Matrices"
    
mcSpec = {
    "expression": None,
    "result":None,
    "constraint": {
        "by_value": None,
        "by_zone": None
    },
    "aggregation": {
        "origins": "+",
        "destinations": "+"
    },
    "type": "MATRIX_CALCULATION"
}

In [2]:
auto_demand_pk = "TRIPS TOTAL AUTO peak"
auto_demand_offpk = "TRIPS TOTAL AUTO offpeak"

trip_am_auto1nt = "trips_am_peak_auto1_nt"
trip_am_auto1t = "trips_am_peak_auto1_toll" 
trip_am_auto2nth = "trips_am_peak_auto2_nt_hov"
trip_am_auto2ntnh = "trips_am_peak_auto2_nt_nh" 
trip_am_auto2th = "trips_am_peak_auto2_toll_hov" 
trip_am_auto2tnh ="trips_am_peak_auto2_toll_nh" 
trip_am_auto3nth ="trips_am_peak_auto3_nt_hov" 
trip_am_auto3ntnh ="trips_am_peak_auto3_nt_nh" 
trip_am_auto3th ="trips_am_peak_auto3_toll_hov" 
trip_am_auto3tnh ="trips_am_peak_auto3_toll_nh" 
trip_md_auto1nt ="trips_midday_auto1_nt" 
trip_md_auto1t ="trips_midday_auto1_toll"
trip_md_auto2nth ="trips_midday_auto2_nt_hov" 
trip_md_auto2ntnh ="trips_midday_auto2_nt_nh" 
trip_md_auto2th ="trips_midday_auto2_toll_hov" 
trip_md_auto2tnh ="trips_midday_auto2_toll_nh" 
trip_md_auto3nth ="trips_midday_auto3_nt_hov" 
trip_md_auto3ntnh ="trips_midday_auto3_nt_nh"
trip_md_auto3th ="trips_midday_auto3_toll_hov" 
trip_md_auto3tnh ="trips_midday_auto3_toll_nh" 

roadClasses = ["Freeway", "Highway", "Arterial"]
matchveh = {"light/medium trucks":"VKT_lmt", "heavy trucks":"VKT_ht", "buses":"VKT_bus"}
vehicleTypes = ["light/medium trucks","heavy trucks","buses"]

In [3]:
sc_2051_BAU_ogs = "2051\\Medium List\\BAU\\BAU_Updated_APTG_2051_GGH_MTP_V106A3_CS222_20200727"

#Network 1
sc_2051_N1_aptg = "2051\\Medium List\\Network 1\\N1_APTG_2051_GGH_MTP_V106A3_CS222_2020821"
sc_2051_N1_luv1 = "2051\\Medium List\\Network 1\\N1_LUV1_2051_GGH_MTP_V106A3_CS222_20200918"

#Network 2
sc_2051_N2_aptg = "2051\\Medium List\\Network 2\\N2_APTG_2051_GGH_MTP_V106A3_CS222_2020821"
sc_2051_N2_luv1 = "2051\\Medium List\\Network 2\\N2_LUV1_2051_GGH_MTP_V106A3_CS222_20200924"

#Network 3
sc_2051_N3_aptg = "2051\\Medium List\\Network 3\\N3_APTG_2051_GGH_MTP_V106A3_CS222_20200901"
sc_2051_N3_luv2 = "2051\\Medium List\\Network 3\\N3_LUV2_2051_GGH_MTP_V106A3_CS222_20201009"

#Network 4
sc_2051_N4_aptg = "2051\\Medium List\\Network 4\\N4_APTG_2051_GGH_MTP_V106A3_CS222_20200911"
sc_2051_N4_luv3 = "2051\\Medium List\\Network 4\\N4_LUV3_2051_GGH_MTP_V106A3_CS222_20201002"

#Network 5
sc_2051_N5_aptg = "2051\\Medium List\\Network 5\\N5_APTG_2051_GGH_MTP_V106A3_HS022_20200918"
sc_2051_N5_luv3 = "2051\\Medium List\\Network 5\\N5_LUV3_2051_GGH_MTP_V106A3_CS222_20201002"

In [4]:
# scens = {"2051_BAU_OGS":[sc_2051_BAU_ogs,[404,405]],
#          "2051_N1_APTG":[sc_2051_N1_aptg,[501,502]],
#         "2051_N2_APTG":[sc_2051_N2_aptg,[601,602]],
#         "2051_N3_APTG":[sc_2051_N3_aptg,[701,702]],
#         "2051_N4_APTG":[sc_2051_N4_aptg,[801,802]],
#         "2051_N5_APTG":[sc_2051_N5_aptg,[901,902]]}

# scens = {"2051_N5_APTG":[sc_2051_N5_aptg,[901,902]]}

# scens = {"2051_N1_LUV1":[sc_2051_N1_luv1,[504,505]],
# "2051_N2_LUV1":[sc_2051_N2_luv1,[604,605]],
# "2051_N4_LUV3":[sc_2051_N4_luv3,[804,805]],
# "2051_N5_LUV3":[sc_2051_N5_luv3,[904,905]],
# "2051_N3_LUV2":[sc_2051_N3_luv2,[704,705]]}

scens = {"2051_N3_LUV2":[sc_2051_N3_luv2,[704,705]]}


truck_vols = ["@volau_mtruck","@volau_ltruck"]
auto_vols = ["@volau1_nt","@volau1_toll",\
             "@volau2_nt_nh","@volau2_nt_hov","@volau2_toll_nh","@volau2_toll_hov",\
            "@volau3_nt_nh","@volau3_nt_hov","@volau3_toll_nh","@volau3_toll_hov"]
hdr_bau_peak = "mf160"
hdr_bau_offpeak = "mf170"

In [5]:
for scen in scens:
    sc_num_am = scens[scen][1][0]
    sc_num_md = scens[scen][1][1]
    sc_am = eb.scenario(sc_num_am)
    sc_md = eb.scenario(sc_num_md)
    scenario_folder = scens[scen][0]
    matrix_folder = r"\\tore-infs01\PWExternal\451010\281829_GGH\GGHM Runs" + "\\" + scenario_folder
    #networks
    ntwk_am = sc_am.get_network()
    ntwk_md = sc_md.get_network()
    clear_matrices(eb,[hdr_bau_peak, hdr_bau_offpeak],"FULL")
    matrix_list = {auto_demand_pk:"",
                    auto_demand_offpk:"",
                    trip_am_auto1nt:"",
                    trip_am_auto1t:"",
                    trip_am_auto2nth:"",
                    trip_am_auto2ntnh:"",
                    trip_am_auto2th:"",
                    trip_am_auto2tnh:"",
                    trip_am_auto3nth:"",
                    trip_am_auto3ntnh:"",
                    trip_am_auto3th:"",
                    trip_am_auto3tnh:"",
                    trip_md_auto1nt:"",
                    trip_md_auto1t:"",
                    trip_md_auto2nth:"",
                    trip_md_auto2ntnh:"",
                    trip_md_auto2th:"",
                    trip_md_auto2tnh:"",
                    trip_md_auto3nth:"",
                    trip_md_auto3ntnh:"",
                    trip_md_auto3th:"",
                    trip_md_auto3tnh:""}
    for matrix_name in matrix_list.keys():
        matrix_list[matrix_name] = eb.available_matrix_identifier('FULL')
        print matrix_name
        create_matrix(matrix_id = matrix_list[matrix_name],
                      matrix_name= matrix_name.replace("skim.","").replace("transit.","").replace("peak","pk").replace(" ","_").strip()[:40],
                      matrix_description = matrix_name)
    
    import_matrices_from_directory(matrix_folder, matrix_list, ".mdf",sc_am)
    
    auto_pk_hr_exp = matrix_list[trip_am_auto1nt] + "+" + matrix_list[trip_am_auto1t] + "+" + \
    matrix_list[trip_am_auto2nth] + "+" + matrix_list[trip_am_auto2ntnh] + "+" + matrix_list[trip_am_auto2th] + "+" + \
    matrix_list[trip_am_auto2tnh] + "+" + matrix_list[trip_am_auto3nth] + "+" + matrix_list[trip_am_auto3ntnh] + "+" + \
    matrix_list[trip_am_auto3th] + "+" + matrix_list[trip_am_auto3tnh]
    
    auto_offpk_hr_exp = matrix_list[trip_md_auto1nt] + "+" + matrix_list[trip_md_auto1t] + "+" + \
    matrix_list[trip_md_auto2nth] + "+" + matrix_list[trip_md_auto2ntnh] + "+" + matrix_list[trip_md_auto2th] + "+" + \
    matrix_list[trip_md_auto2tnh] + "+" + matrix_list[trip_md_auto3nth] + "+" + matrix_list[trip_md_auto3ntnh] + "+" + \
    matrix_list[trip_md_auto3th] + "+" + matrix_list[trip_md_auto3tnh]
    
    vkt_am = {"Freeway":{"VKT_tot":0, "VHT_tot":0,"VKT_lmt":0, "VKT_ht":0, "VKT_bus":0},
          "Highway":{"VKT_tot":0, "VHT_tot":0,"VKT_lmt":0, "VKT_ht":0, "VKT_bus":0},
          "Arterial":{"VKT_tot":0, "VHT_tot":0,"VKT_lmt":0, "VKT_ht":0, "VKT_bus":0}}
    vkt_md = {"Freeway":{"VKT_tot":0, "VHT_tot":0,"VKT_lmt":0, "VKT_ht":0, "VKT_bus":0},
          "Highway":{"VKT_tot":0, "VHT_tot":0,"VKT_lmt":0, "VKT_ht":0, "VKT_bus":0},
          "Arterial":{"VKT_tot":0, "VHT_tot":0,"VKT_lmt":0, "VKT_ht":0, "VKT_bus":0}}
    
    for link in ntwk_am.links():
        vdf = link.volume_delay_func
        roadClass = None
        speed = link.data2
        time = link.auto_time
        if (vdf > 0 and vdf < 20) and ntwk_am.mode("I") in link.modes:
            roadClass = "Freeway"
        elif (vdf >= 20 and vdf < 30) and ntwk_am.mode("I") in link.modes:
            roadClass = "Highway"
        elif (vdf == 30 or vdf == 31 or vdf == 40 or vdf == 50) and ntwk_am.mode("I") in link.modes:
            roadClass = "Arterial"

        if roadClass is not None:
            tot_vol = 0.0
            lm_truck_vol = 0.0
            h_truck_vol = link["@volau_htruck"]
            tot_vol+= link["@volau_htruck"]

            b_vol = link["@ca_trveh_l"]
            tot_vol+= link["@ca_trveh_l"]

            for vol in auto_vols:
                tot_vol += link[vol]
            for vol in truck_vols:
                tot_vol += link[vol]
                lm_truck_vol += link[vol]

            

            vkt_am[roadClass]["VKT_tot"] += tot_vol*link.length
            vkt_am[roadClass]["VHT_tot"] += tot_vol*time/60
            vkt_am[roadClass]["VKT_lmt"] += lm_truck_vol*link.length
            vkt_am[roadClass]["VKT_ht"] += h_truck_vol*link.length
            vkt_am[roadClass]["VKT_bus"] += b_vol*link.length
        
    for link in ntwk_md.links():
        vdf = link.volume_delay_func
        roadClass = None
        speed = link.data2
        if (vdf > 0 and vdf < 20) and ntwk_md.mode("I") in link.modes:
            roadClass = "Freeway"
        elif (vdf >= 20 and vdf < 30) and ntwk_md.mode("I") in link.modes:
            roadClass = "Highway"
        elif (vdf == 30 or vdf == 31 or vdf == 40 or vdf == 50) and ntwk_md.mode("I") in link.modes:
            roadClass = "Arterial"

        if roadClass is not None:
            tot_vol = 0.0
            lm_truck_vol = 0.0
            h_truck_vol = link["@volau_htruck"]
            tot_vol+= link["@volau_htruck"]

            b_vol = link["@ca_trveh_l"]
            tot_vol+= link["@ca_trveh_l"]

            for vol in auto_vols:
                tot_vol += link[vol]
            for vol in truck_vols:
                tot_vol += link[vol]
                lm_truck_vol += link[vol]

            time = link.auto_time

            vkt_md[roadClass]["VKT_tot"] += tot_vol*link.length
            vkt_md[roadClass]["VHT_tot"] += tot_vol*time/60
            vkt_md[roadClass]["VKT_lmt"] += lm_truck_vol*link.length
            vkt_md[roadClass]["VKT_ht"] += h_truck_vol*link.length
            vkt_md[roadClass]["VKT_bus"] += b_vol*link.length
            
    outfile = r"C:\Users\bcrane\Documents\GGH - Medium List\RWDI_MOE_ML_{}_AM_20201014.csv".format(scen)
    moef = open(outfile, 'w')
    calc = deepcopy(mcSpec)
    calc["expression"] = matrix_list[auto_demand_pk]
    pk_pd_dem = matrix_calculator(calc)["result"]
    
    calc2 = deepcopy(mcSpec)
    calc2["expression"] = auto_pk_hr_exp
    pk_hr_dem = matrix_calculator(calc2)["result"]
    
    moef.write("Auto Peak Period Total Demand,{0:.4f}\n".format(pk_pd_dem))
    moef.write("Auto Peak Hour Total Demand,{0:.4f}\n".format(pk_hr_dem))
    moef.write("Hours,7\n")
    
    for roadClass in roadClasses:
        moef.write("Total {} VKT,".format(roadClass))
        moef.write("{0:.4f}\n".format(vkt_am[roadClass]["VKT_tot"]))
    for roadClass in roadClasses:
        moef.write("Average {} Speed,".format(roadClass))
        moef.write("{0:.4f}\n".format(vkt_am[roadClass]["VKT_tot"]/vkt_am[roadClass]["VHT_tot"]))
    for roadClass in roadClasses:
        for veh in vehicleTypes:
            moef.write("% {} Vehicles that are {},".format(roadClass,veh))
            moef.write("{0:.4f}\n".format(vkt_am[roadClass][matchveh[veh]]/vkt_am[roadClass]["VKT_tot"]))
    moef.close()
    
    outfile = r"C:\Users\bcrane\Documents\GGH - Medium List\RWDI_MOE_ML_{}_MD_20201014.csv".format(scen)
    moef = open(outfile, 'w')
    calc = deepcopy(mcSpec)
    calc["expression"] = matrix_list[auto_demand_offpk]
    offpk_pd_dem = matrix_calculator(calc)["result"]
    
    calc2 = deepcopy(mcSpec)
    calc2["expression"] = auto_offpk_hr_exp
    offpk_hr_dem = matrix_calculator(calc2)["result"]
    
    moef.write("Auto Peak Period Total Demand,{0:.4f}\n".format(offpk_pd_dem))
    moef.write("Auto Peak Hour Total Demand,{0:.4f}\n".format(offpk_hr_dem))
    moef.write("Hours,13\n")
    
    for roadClass in roadClasses:
        moef.write("Total {} VKT,".format(roadClass))
        moef.write("{0:.4f}\n".format(vkt_md[roadClass]["VKT_tot"]))
    for roadClass in roadClasses:
        moef.write("Average {} Speed,".format(roadClass))
        moef.write("{0:.4f}\n".format(vkt_md[roadClass]["VKT_tot"]/vkt_md[roadClass]["VHT_tot"]))
    for roadClass in roadClasses:
        for veh in vehicleTypes:
            moef.write("% {} Vehicles that are {},".format(roadClass,veh))
            moef.write("{0:.4f}\n".format(vkt_md[roadClass][matchveh[veh]]/vkt_md[roadClass]["VKT_tot"]))
    moef.close()

trips_midday_auto2_toll_nh
trips_am_peak_auto2_nt_hov
trips_midday_auto3_toll_hov
trips_midday_auto2_nt_nh
trips_midday_auto3_nt_nh
trips_midday_auto2_toll_hov
trips_am_peak_auto1_nt
TRIPS TOTAL AUTO offpeak
trips_am_peak_auto2_nt_nh
trips_midday_auto1_toll
trips_midday_auto2_nt_hov
trips_am_peak_auto3_toll_hov
trips_midday_auto3_nt_hov
trips_am_peak_auto3_nt_hov
trips_am_peak_auto2_toll_nh
trips_midday_auto3_toll_nh
TRIPS TOTAL AUTO peak
trips_am_peak_auto3_nt_nh
trips_midday_auto1_nt
trips_am_peak_auto2_toll_hov
trips_am_peak_auto3_toll_nh
trips_am_peak_auto1_toll
Imported matrix mf8
Imported matrix mf17
Imported matrix mf7
Imported matrix mf22
Imported matrix mf2
Imported matrix mf9
Imported matrix mf20
Imported matrix mf15
Imported matrix mf14
Imported matrix mf18
Imported matrix mf12
Imported matrix mf21
Imported matrix mf19
Imported matrix mf10
Imported matrix mf11
Imported matrix mf4
Imported matrix mf6
Imported matrix mf1
Imported matrix mf13
Imported matrix mf5
Imported matrix

In [13]:
vkt_md

{'Arterial': {'VHT_tot': 108977.73036897388,
  'VKT_bus': 36183.14231549603,
  'VKT_ht': 650089.7952342913,
  'VKT_lmt': 668100.2127851765,
  'VKT_tot': 5284220.258164351},
 'Freeway': {'VHT_tot': 97235.67078831905,
  'VKT_bus': 5523.772805254266,
  'VKT_ht': 2238951.6222152878,
  'VKT_lmt': 1363806.0252735603,
  'VKT_tot': 7948400.50806382},
 'Highway': {'VHT_tot': nan,
  'VKT_bus': 2645.96873220646,
  'VKT_ht': 957008.2478030922,
  'VKT_lmt': 908643.1324533466,
  'VKT_tot': 5531049.447290685}}